In [1]:
 import os
 import shutil

 # Input folder containing the images
 input_dir = r"/kaggle/input/skin-disease-dataset/dataset/train"
 # Output folder for renamed images
 output_dir = r"/kaggle/working/renamed_train"

 # Ensure the output directory exists
 os.makedirs(output_dir, exist_ok=True)

 # Dictionary to track counts for each class
 class_counts = {}

 # Traverse through each subdirectory
 for root, dirs, files in os.walk(input_dir):
     for file_name in files:
         # Full path of the image
         img_path = os.path.join(root, file_name)

         # Skip non-image files
         if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
             print(f"Skipping non-image file: {file_name}")
             continue

         # Get the folder name (class name) as the class identifier
         class_name = os.path.basename(root)

         # Initialize or increment the count for this class
         if class_name not in class_counts:
             class_counts[class_name] = 1
         else:
             class_counts[class_name] += 1

         # Generate new file name in the format ClassName(Count).Extension
         count = class_counts[class_name]
         ext = os.path.splitext(file_name)[1]  # Get file extension
         new_name = f"{class_name}({count}){ext}"
         new_path = os.path.join(output_dir, new_name)

         # Copy and rename the file to the output directory
         shutil.copy(img_path, new_path)

 # Print the total number of images for each class
 print("\nImage counts by class:")
 for class_name, count in class_counts.items():
     print(f"{class_name}: {count} images")

 print("\nRenaming and consolidation complete!")


Image counts by class:
Eczema: 999 images
Melanoma: 1000 images
Basal Cell: 1000 images
Seborrheic: 1000 images
Atopic Dermatitis: 1000 images
Melanocytic: 1000 images
Benign Keratosis: 1201 images
Warts Molluscum: 1000 images
Psoriasis: 1000 images
Tinea Ringworms Candidiasis: 990 images

Renaming and consolidation complete!


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import cv2

# Define class mapping
class_mapping = {
    "Seborrheic": 0,
    "Melanocytic": 1,
    "Melanoma": 2,
    "Eczema": 3,
    "Basal_Cell": 4,  # New class added
}

# Reverse mapping for class names
label_to_class = {v: k for k, v in class_mapping.items()}

# Preprocess images: resize and normalize
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: {image_path} could not be loaded.")
        return None

    resized_image = cv2.resize(image, (299, 299))  # Resize for InceptionResNetV2
    img_normalized = resized_image.astype('float32') / 255.0  # Normalize to [0, 1]
    return img_normalized

# Load dataset
def load_data_from_single_folder(folder):
    images = []
    labels = []

    image_files = sorted([f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    for image_name in image_files:
        image_path = os.path.join(folder, image_name)
        label = image_name.split('(')[0].strip().replace(' ', '_')  # Extract class name
        
        if label in class_mapping:
            label_index = class_mapping[label]
        else:
            continue

        preprocessed_image = preprocess_image(image_path)
        if preprocessed_image is not None:
            images.append(preprocessed_image)
            labels.append(label_index)

    print(f"Loaded {len(images)} images and {len(labels)} labels.")
    return np.array(images), np.array(labels)

# Load train and test data
train_folder = r'/kaggle/working/renamed_train'
X_train, y_train = load_data_from_single_folder(train_folder)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=5)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=np.argmax(y_train, axis=1)
)

# Data augmentation for class balancing
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define the model using InceptionResNetV2
# base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
# base_model.trainable = False  # Freeze base model

# model = tf.keras.Sequential([
#     base_model, 
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(5, activation='softmax')  # Output layer (5 classes)
# ])

# # Unfreeze last few layers of base model for fine-tuning
# for layer in base_model.layers[-20:]:
#     layer.trainable = True

# # Compile model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# print(model.summary())

# # Callbacks for early stopping and learning rate reduction
# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
#     ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
# ]

# # Train the model
# epochs = 50
# batch_size = 16
# history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
#                     validation_data=(X_test, y_test), callbacks=callbacks, shuffle=True)

# # Predict on the test set
# y_test_pred = model.predict(X_test)
# y_test_pred_classes = np.argmax(y_test_pred, axis=1)
# y_test_actual_classes = np.argmax(y_test, axis=1)

# # Confusion matrix
# test_conf_matrix = confusion_matrix(y_test_actual_classes, y_test_pred_classes)
# print(f"Test Confusion Matrix:\n{test_conf_matrix}")

# # Calculate accuracy
# test_accuracy = np.mean(y_test_pred_classes == y_test_actual_classes)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# # Print first 10 predictions
# for i in range(10):
#     predicted_class = label_to_class[y_test_pred_classes[i]]
#     actual_class = label_to_class[y_test_actual_classes[i]]
#     print(f"Predicted: {predicted_class}, Actual: {actual_class}")


Loaded 4999 images and 4999 labels.


In [3]:
# base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
# base_model.trainable = False  # Freeze base model

# model = tf.keras.Sequential([
#     base_model, 
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(5, activation='softmax')  # Output layer (5 classes)
# ])

# # Unfreeze last few layers of base model for fine-tuning
# for layer in base_model.layers[-20:]:
#     layer.trainable = True

# # Compile model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# print(model.summary())

# # Callbacks for early stopping and learning rate reduction

# # Train the model
# epochs = 50
# batch_size = 16
# history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
#                     validation_data=(X_test, y_test))

# # Predict on the test set
# y_test_pred = model.predict(X_test)
# y_test_pred_classes = np.argmax(y_test_pred, axis=1)
# y_test_actual_classes = np.argmax(y_test, axis=1)

# # Confusion matrix
# test_conf_matrix = confusion_matrix(y_test_actual_classes, y_test_pred_classes)
# print(f"Test Confusion Matrix:\n{test_conf_matrix}")

# # Calculate accuracy
# test_accuracy = np.mean(y_test_pred_classes == y_test_actual_classes)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# # Print first 10 predictions
# for i in range(10):
#     predicted_class = label_to_class[y_test_pred_classes[i]]
#     actual_class = label_to_class[y_test_actual_classes[i]]
#     print(f"Predicted: {predicted_class}, Actual: {actual_class}")


In [4]:


# from tensorflow.keras.applications import DenseNet121
# base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

# model = tf.keras.Sequential([
#     base_model, 
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(5, activation='softmax')  # Output layer (5 classes)
# ])

# # Unfreeze last few layers of base model for fine-tuning
# for layer in base_model.layers[-20:]:
#     layer.trainable = True

# # Compile model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# print(model.summary())

# # Callbacks for early stopping and learning rate reduction

# # Train the model
# epochs = 50
# batch_size = 16
# history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
#                     validation_data=(X_test, y_test))

# # Predict on the test set
# y_test_pred = model.predict(X_test)
# y_test_pred_classes = np.argmax(y_test_pred, axis=1)
# y_test_actual_classes = np.argmax(y_test, axis=1)

# # Confusion matrix
# test_conf_matrix = confusion_matrix(y_test_actual_classes, y_test_pred_classes)
# print(f"Test Confusion Matrix:\n{test_conf_matrix}")

# # Calculate accuracy
# test_accuracy = np.mean(y_test_pred_classes == y_test_actual_classes)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# # Print first 10 predictions
# for i in range(10):
#     predicted_class = label_to_class[y_test_pred_classes[i]]
#     actual_class = label_to_class[y_test_actual_classes[i]]
#     print(f"Predicted: {predicted_class}, Actual: {actual_class}")


In [5]:

from tensorflow.keras.applications import DenseNet121
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

model = tf.keras.Sequential([
    base_model, 
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax')  # Output layer (5 classes)
])

# Unfreeze last few layers of base model for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

# Callbacks for early stopping and learning rate reduction
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]

# Train the model
epochs = 50
batch_size = 16
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=callbacks, shuffle=True)

# Predict on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_actual_classes = np.argmax(y_test, axis=1)

# Confusion matrix
test_conf_matrix = confusion_matrix(y_test_actual_classes, y_test_pred_classes)
print(f"Test Confusion Matrix:\n{test_conf_matrix}")

# Calculate accuracy
test_accuracy = np.mean(y_test_pred_classes == y_test_actual_classes)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Print first 10 predictions
for i in range(10):
    predicted_class = label_to_class[y_test_pred_classes[i]]
    actual_class = label_to_class[y_test_actual_classes[i]]
    print(f"Predicted: {predicted_class}, Actual: {actual_class}")


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)             │ (None, 9, 9, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,103,429 (27.10 MB)

 Trainable params: 7,019,781 (26.78 MB)

 Non-trainable params: 83,648 (326.75 KB)

None
Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 325s 672ms/step - accuracy: 0.7780 - loss: 0.5706 - val_accuracy: 0.8980 - val_loss: 0.2771 - learning_rate: 1.0000e-04
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 47s 186ms/step - accuracy: 0.9482 - loss: 0.1458 - val_accuracy: 0.9170 - val_loss: 0.2238 - learning_rate: 1.0000e-04
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 46s 185ms/step - accuracy: 0.9746 - loss: 0.0805 - val_accuracy: 0.9470 - val_loss: 0.1556 - learning_rate: 1.0000e-04
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 46s 184ms/step - accuracy: 0.9868 - loss: 0.0469 - val_accuracy: 0.9070 - val_loss: 0.2832 - learning_rate: 1.0000e-04
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 46s 184ms/step - accuracy: 0.9793 - loss: 0.0582 - val_accuracy: 0.9220 - val_loss: 0.2483 - learning_rate: 1.0000e-04
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 46s 184ms/step - accuracy: 0.9837 - loss: 0.0578 - val_accuracy: 0.8370 - val_loss: 0.5700 - learning_rate: 1.0000e-04
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [6]:
model.save("densenet_model.h5")

In [7]:
model.save_weights("densenet.weights.h5")